In [ ]:
import numpy as np
import os
from pymongo import MongoClient
import utils.const as const
import re
import demoji

In [ ]:
client = MongoClient()
client.list_database_names() # ['admin', 'local']

In [ ]:
db=const.grpId
collection=const.PostCollection
data='data.txt'
count='count.txt'
id_file='id.txt'
myCollect=client[db][collection]


In [ ]:
myCollect=client[db][collection]
dataFp=open(data,'w')
countFp=open(count,'w')
idFp=open(id_file,'w')

In [ ]:
def process(s):
    s=s.replace('\n',' ')  ## new line
    
    s = re.sub('https?:\S*', ' ', s, flags=re.MULTILINE) ## links
    
    s=re.sub('[#$]+',' ',s) ## characters that are not required
    
    s=demoji.replace_with_desc(s, "$$") ## emoji graphic
    s=s.replace('$$',' ') 
    
    s=re.sub('\s+', ' ',s) ## extra space
    
    return s;

In [ ]:
# post msg
c=0
for post in myCollect.find({}):
    m=''
    if 'message' in post:
        m=post['message']
        
    dataFp.write(process(m)+'\n')
    idFp.write(post['_id']+'\n')
    
    c=c+1
    
countFp.write(str(c)+'\n')

In [ ]:
# post comment
# c=0
# for post in myCollect.find({}):
#     if 'comments' in post:
#         for comment in post['comments']:
#             m=''
#             if 'message' in comment:
#                 m=comment['message']
                
#             dataFp.write(process(m)+'\n')
            
#             idFp.write(post['_id']+' '+comment['_id']+'\n')
#             c=c+1
# countFp.write(str(c)+'\n')

In [ ]:
# post subcomment 
# c=0
# for post in myCollect.find({}):
#     if 'comments' in post:
#         for comment in post['comments']:
#             if 'subComments' in comment:
#                 for subComment in comment['subComments']:
#                     m=''
#                     if 'message' in subComment:
#                         m=subComment['message']
#                     dataFp.write(process(m)+'\n')
#                     idFp.write(post['_id']+' '+comment['_id']+' '+subComment['_id']+' '+subComment['created_time']+' '+subComment['lang']+'\n')
#                     c=c+1
# countFp.write(str(c)+'\n')

In [ ]:
dataFp.close()
idFp.close()
countFp.close()

# Following code can be used to create csv file for transaltion using Goggle sheet

In [ ]:
import xlsxwriter
import xlrd

row_limit=20000
outputFile='dataOut.xlsx'
inputFile='dataIn.xlsx'

In [ ]:
fp=open(data,'r')
lines=fp.readlines()
fp.close()

l=len(lines)
print(l)

colNum=(int)(l/row_limit)
if colNum*row_limit != l:
    colNum=colNum+1
print(colNum)

workbook = xlsxwriter.Workbook(outputFile)
worksheet = workbook.add_worksheet()

rowCount=0

for i in range(row_limit):
    if i*colNum>=l:
        break;
        
    st=''
    for j in range(colNum):
        if i*colNum + j < l:
            st+=lines[i*colNum + j][:-1]
            st+=' $ '
        else:
            break
    if len(st)==0:
        st+=' '
    worksheet.write(i,0,st)
    rowCount+=1

print('rows in sheet '+str(rowCount))

workbook.close()

## now upload outputFile in google sheet and translate it then copy the result in inputFile

In [ ]:
fp=open(data,'w')

colNum=(int)(l/row_limit)
if colNum*row_limit != l:
    colNum=colNum+1

workbook = xlrd.open_workbook(inputFile)
sheet = workbook.sheet_by_index(0)

print('rows in sheet '+str(sheet.nrows))

for i in range(sheet.nrows):
    st=sheet.cell_value(i, 0)
    arr=st.split('$')
    
    if len(arr) < colNum:
        print(i,len(arr))
    
    while len(arr)<colNum+1: ### +1 because of second last line of above cell does not have st[:-2] but st
        arr.append('')
    
    for k in arr[0:colNum]:
        fp.write(k.strip()+"\n")

fp.close()